In [1]:
## coding=utf-8

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

    

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

Student: Louis Montaut

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is what allows the agent to select an action according to an incoming state <code>s</code>, thanks to its policy <code>learned_act()</code>. Because we are using a DQN, which is based on the estimation of the Q-function and then using a greedy policy related to this Q function, if we don't inforce some kind of exploration, the agent will focus on always doing the same kind of behavior. Therefore, at each step, there is a probability <code>epsilon</code> that the agent takes a random action, allowing it to explore its environment.<br>
In order to have a good policy, the optimal thing is to decrease the value of <code>epsilon</code> through time and end up with <code>epsilon=0</code> (this is not implemented yet).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size)) # this is where the reward is stored
        self.position = np.zeros((grid_size,grid_size)) # useful for the state

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256 # Cheese is red
        b[self.board < 0, 2] = 256 # Poison is blue
        b[self.x,self.y,:] = 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13 # size of the grid environment
T=200 # max time that the mouse has to live
temperature = 0.3 # regulating the flow of cheese/poison in the environment
epochs_train = 100 # set small when debugging
epochs_test = 100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The <code>position</code> array is used to determine the state it is in, a.k.a the 25 cells around it that it sees. This state is, of course, primordial to know for our reinforcement learning problem.<br> <br>
The <code>board</code> array stores the rewards that are layed out on the board. To build it, we first generate two random arrays: <code>bonus</code> and <code>malus</code>. Both of these arrays are of the size of the grid and are generated by two bernouilli laws (on each cell of the grid), $0.5*B(temperature)$ for <code>bonus</code> and $-1*B(temperature)$ for <code>malus</code> (where $B(p)$ is a bernouilli law of probability of success p).<br>
Finally we assemble these two arrays into one by saying that if a cell has both 0.5 and -1 values, it keeps only the 0.5 bonus value.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(self.n_action)
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix='', display_score=True):

    # Number of won games
    score = 0

    for e in range(epochs):
        # Let's use the code at the beginning, under section "the game"

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # print(state.shape) 
        # state is of shape (5,5,2), and in general NxNx2
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e+1))

        # Update stats
        score = score + win-lose
        
        if display_score:
            print("Epoch: {}, win/lose count {}/{}. Average score ({})"
            .format(e+1,win, lose, score/(1+e)))

    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,10,prefix='random')
HTML(display_videos('random0.mp4'))

Epoch: 1, win/lose count 7.5/9.0. Average score (-1.5)
Epoch: 2, win/lose count 9.5/14.0. Average score (-3.0)
Epoch: 3, win/lose count 12.0/18.0. Average score (-4.0)
Epoch: 4, win/lose count 11.5/12.0. Average score (-3.125)
Epoch: 5, win/lose count 7.5/9.0. Average score (-2.8)
Epoch: 6, win/lose count 3.5/15.0. Average score (-4.25)
Epoch: 7, win/lose count 7.0/16.0. Average score (-4.928571428571429)
Epoch: 8, win/lose count 11.5/12.0. Average score (-4.375)
Epoch: 9, win/lose count 3.5/12.0. Average score (-4.833333333333333)
Epoch: 10, win/lose count 10.5/22.0. Average score (-5.5)
Final score: -5.5


In [10]:
# High temperature
temperature = 0.8
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,10,prefix='random')
HTML(display_videos('random0.mp4'))

Epoch: 1, win/lose count 19.0/7.0. Average score (12.0)
Epoch: 2, win/lose count 18.5/11.0. Average score (9.75)
Epoch: 3, win/lose count 16.0/12.0. Average score (7.833333333333333)
Epoch: 4, win/lose count 31.0/12.0. Average score (10.625)
Epoch: 5, win/lose count 33.0/6.0. Average score (13.9)
Epoch: 6, win/lose count 31.0/6.0. Average score (15.75)
Epoch: 7, win/lose count 27.0/20.0. Average score (14.5)
Epoch: 8, win/lose count 31.0/12.0. Average score (15.0625)
Epoch: 9, win/lose count 22.5/17.0. Average score (14.0)
Epoch: 10, win/lose count 28.0/10.0. Average score (14.4)
Final score: 14.4


In [11]:
# Low temperature
temperature = 0.1
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,10,prefix='random')
HTML(display_videos('random0.mp4'))

Epoch: 1, win/lose count 3.5/4.0. Average score (-0.5)
Epoch: 2, win/lose count 6.5/4.0. Average score (1.0)
Epoch: 3, win/lose count 3.5/5.0. Average score (0.16666666666666666)
Epoch: 4, win/lose count 3.0/7.0. Average score (-0.875)
Epoch: 5, win/lose count 4.0/6.0. Average score (-1.1)
Epoch: 6, win/lose count 1.0/4.0. Average score (-1.4166666666666667)
Epoch: 7, win/lose count 6.5/3.0. Average score (-0.7142857142857143)
Epoch: 8, win/lose count 2.5/3.0. Average score (-0.6875)
Epoch: 9, win/lose count 5.5/5.0. Average score (-0.5555555555555556)
Epoch: 10, win/lose count 2.5/7.0. Average score (-0.95)
Final score: -0.95


As expected, a random behavior.

Warning: as explained before in Question 2, the action of temperature is peculiar. If the temperature is high, there is an increased probability that in each cell there is either a nice cheese or a dirty poison but since we have the condition <code>malus[bonus>0]=0</code> in the code, when the temperature rises, there is more cheese on the board and less poison. When the temperature decreases, the simply is less happening on the board. 

A low temperature will be good to test the exploration ability of the mouse.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ 1°) Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

2°) Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
3°) Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Question 5: answer__

1°) Since $T=\infty$, $Q^{\pi}(s,a) = E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r_{t}|s_{0}=s, a_{0}=a]$, where $r_{t}=r(s_{t},a_{t})$ for simplicity.Therefore:

\begin{equation*}
Q^{\pi}(s,a) = r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r_{t+1}|s_{0}=s, a_{0}=a]
\end{equation*}

Let's use $(s_{1},a_{1})$ as being a complete event system:
\begin{equation*}
Q^{\pi}(s,a) = r(s,a) +\sum_{s',a'}p(s_{1}, a_{1}=s', a'|s_{0}, a_{0}=s, a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r_{t+1}|s_{0}=s, a_{0}=a,s_{1}=s', a_{1}=a']
\end{equation*}

Let's use the Markov property to get rid of $s_{0}$ and $a_{0}$ in the expectation:
\begin{equation*}
Q^{\pi}(s,a) = r(s,a) +\sum_{s',a'}p(s_{1}, a_{1}=s', a'|s_{0}, a_{0}=s, a) \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r_{t+1}|s_{1}=s', a_{1}=a'] \\
= r(s,a) +\sum_{s',a'}p(s_{1}, a_{1}=s', a'|s_{0}, a_{0}=s, a) \gamma Q^{\pi}(s',a') \\
= E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')]
\end{equation*}

2°) If we take the max of the expression above:

\begin{equation*}
    Q^{*}(s,a) = max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')] \\
    = r(s,a) +max_{\pi}\sum_{s',a'}p(s_{1}, a_{1}=s', a'|s_{0}, a_{0}=s, a) \gamma Q^{\pi}(s',a')
\end{equation*}

And we know that $p(s_{1}, a_{1}=s', a'|s_{0}, a_{0}=s, a)=p(s_{1}=s'|s_{0}, a_{0}=s, a)\pi(a_{1}= a'|s_{1}=s')$, therefore:

\begin{equation*}
    Q^{*}(s,a) = max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma Q^{\pi}(s',a')] \\
    = r(s,a) +max_{\pi}\sum_{s',a'}p(s_{1}=s'|s_{0}, a_{0}=s, a)\pi(a_{1}= a'|s_{1}=s') \gamma Q^{\pi}(s',a') \\
    = r(s,a) +\gamma \sum_{s'}p(s_{1}=s'|s_{0}, a_{0}=s, a)max_{\pi}\sum_{a'}\pi(a_{1}= a'|s_{1}=s') Q^{\pi}(s',a')
\end{equation*}

Since $\sum_{a'}\pi(a_{1}= a'|s_{1}=s')=1$, we have: $\sum_{a'}\pi(a_{1}= a'|s_{1}=s') Q^{\pi}(s',a') \leqslant max_{a'}Q^{\pi}(s',a')$. <br>Therefore:

\begin{equation*}
    Q^{*}(s,a) = r(s,a) +\gamma \sum_{s'}p(s_{1}=s'|s_{0}, a_{0}=s, a)max_{\pi} max_{a'} Q^{\pi}(s',a') \\
    = r(s,a) +\gamma \sum_{s'}p(s_{1}=s'|s_{0}, a_{0}=s, a) max_{a'} Q^{*}(s',a')
\end{equation*}

Thus:
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

3°) Finally the loss:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
is coherent with the equation in 2°).

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            # if the max_memory has been achieved, remove the oldest element 
            self.memory.pop(0)
            self.memory.append(m)

    def random_access(self):
        if len(self.memory) == 0:
            # if memory has no element, we return None
            return None
        else :
            i = np.random.randint(len(self.memory))
            m = self.memory[i]
            return m

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epochs,prefix='',display_loss=True):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e+1))
            
        if display_loss:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                  .format(e+1, epochs, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.layers import Flatten

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # We first estimate the Q-value thanks to our model
        q = self.model.predict(s.reshape(1,*s.shape))
        action = np.argmax(q)
        return action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        # In remember, we put (s,s',a,r,game_over)
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size,5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # We construct the target for the Q-function
        for i in range(self.batch_size):
            m = self.memory.random_access()
            
            state,next_state,action,reward,game_over = m
            
            input_states[i] = state
            target_q[i] = self.model.predict(state.reshape(1,*state.shape))
            if game_over_:
                target_q[i, action] = reward
            else:
                ns = next_state.reshape(1,*next_state.shape)
                target_q[i, action] = reward + self.discount*np.max(self.model.predict(ns)) 
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC1(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC1, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential([Flatten(input_shape=(5, 5, self.n_state)),
            Dense(20, activation="relu"),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

Recap: training is fixed at 100 epochs, test fixed at 100 epochs. I will use the test function while testing different parameters to see how they influence the performance.

In [77]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)

In [17]:
agent = DQN_FC1(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
print('Training the mouse...')
train(agent,env,epochs_train,prefix='fc_train',display_loss=True)
print('...Done.')
print('Testing the mouse:')
test(agent,env,epochs_test,prefix='fc_test',display_score=True)
HTML(display_videos('fc_test20.mp4'))

Training the mouse...
Epoch 001/100 | Loss 0.2665 | Win/lose count 8.5/6.0 (2.5)
Epoch 002/100 | Loss 0.6119 | Win/lose count 6.0/2.0 (4.0)
Epoch 003/100 | Loss 1.2686 | Win/lose count 1.0/1.0 (0.0)
Epoch 004/100 | Loss 1.8114 | Win/lose count 4.5/5.0 (-0.5)
Epoch 005/100 | Loss 1.8151 | Win/lose count 2.5/4.0 (-1.5)
Epoch 006/100 | Loss 1.9076 | Win/lose count 6.0/3.0 (3.0)
Epoch 007/100 | Loss 1.8440 | Win/lose count 8.0/2.0 (6.0)
Epoch 008/100 | Loss 1.7440 | Win/lose count 6.0/3.0 (3.0)
Epoch 009/100 | Loss 1.8758 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/100 | Loss 1.9324 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/100 | Loss 2.0597 | Win/lose count 3.0/6.0 (-3.0)
Epoch 012/100 | Loss 1.8506 | Win/lose count 7.0/2.0 (5.0)
Epoch 013/100 | Loss 1.8765 | Win/lose count 7.0/5.0 (2.0)
Epoch 014/100 | Loss 1.8679 | Win/lose count 5.5/4.0 (1.5)
Epoch 015/100 | Loss 1.8979 | Win/lose count 4.5/0 (4.5)
Epoch 016/100 | Loss 1.7811 | Win/lose count 3.0/3.0 (0.0)
Epoch 017/100 | Loss 1.8193 | Win

Epoch: 36, win/lose count 9.0/1.0. Average score (10.291666666666666)
Epoch: 37, win/lose count 13.0/2.0. Average score (10.31081081081081)
Epoch: 38, win/lose count 7.5/0. Average score (10.236842105263158)
Epoch: 39, win/lose count 10.5/2.0. Average score (10.192307692307692)
Epoch: 40, win/lose count 16.5/1.0. Average score (10.325)
Epoch: 41, win/lose count 18.0/1.0. Average score (10.487804878048781)
Epoch: 42, win/lose count 11.5/0. Average score (10.511904761904763)
Epoch: 43, win/lose count 13.0/3.0. Average score (10.5)
Epoch: 44, win/lose count 6.0/0. Average score (10.397727272727273)
Epoch: 45, win/lose count 14.5/0. Average score (10.488888888888889)
Epoch: 46, win/lose count 17.0/2.0. Average score (10.58695652173913)
Epoch: 47, win/lose count 10.0/1.0. Average score (10.553191489361701)
Epoch: 48, win/lose count 10.5/1.0. Average score (10.53125)
Epoch: 49, win/lose count 13.0/1.0. Average score (10.561224489795919)
Epoch: 50, win/lose count 9.0/3.0. Average score (10.47

Not bad ! But once the close environment of the mouse is empty of cheese, it oscillates and stops improving. We will work on this later, in the exploration question. Let's try different things regarding the model. I tried changing the different parameters. I tried a one layer model, a two layers one and a three layers one and found that the one layer one performed the best. I tried changing the learning rate but decreasing it makes the learning rate extra-slow (I tried 0.05 and it was already too slow) and increasing it doesn't improve the win/loss count. I found that increasing the batch size a bit increases the performance of the model. <br>
The way I have done my test **is not a grid search**, therefore it is absolutely not optimal. But anyway, here are the results of my tests:

**Let's try changing the model:**

In [18]:
class DQN_FC2(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC2, self).__init__( *args,**kwargs)
        
        # NN Model
        # 2 hidden-layers
        model = Sequential([Flatten(input_shape=(5, 5, self.n_state)),
            Dense(30, activation="relu"),
            Dense(15, activation="relu"),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [20]:
class DQN_FC3(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC3, self).__init__( *args,**kwargs)
        
        # NN Model
        # 3 hidden-layers
        model = Sequential([Flatten(input_shape=(5, 5, self.n_state)),
            Dense(30, activation="relu"),
            Dense(30, activation="relu"),
            Dense(15, activation="relu"),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

Tests made with the following parameters **fixed** <br>
Learning Rate: 0.1, Memory: 2000, Batch Size: 32 

| Model used | Averaged Test Score (Over 100 epochs) |
|------------|---------------------------------------|
| **DQN_FC1**| **9.81**                              |
| DQN_FC2    | 4.23                                  |
| DQN_FC3    | 5.93                                  |

**Impact of memory size**

Tests made with the following parameters **fixed** <br>
Model used: DQN_FC1, Learning Rate: 0.1, Batch Size: 32 

| Memory | Averaged Test Score (Over 100 epochs) |
|--------|---------------------------------------|
| 250    | 9.28                                  |
| 500    | 9.14                                  |
| 1000   | 8.70                                  |
|**2000**| **9.81**                              |
| 5000   | 7.86                                  |

**Impact of batch size**

Tests made with the following parameters **fixed** <br>
Model used: DQN_FC1, Memory: 2000, Learning rate: 0.1 

| Batch Size | Averaged Test Score (Over 100 epochs) |
|------------|---------------------------------------|
| 32         | 9.81                                  |
| **64**     | **10.405**                            |
| 128        | 8.38                                  |
| 256        | 10.905 (Way too long to train)        |

**Impact of learning rate**

Tests made with the following parameters **fixed** <br>
Model used: DQN_FC1, Memory: 2000, Batch Size: 32 

| Learning Rate | Averaged Test Score (Over 100 epochs) |
|---------------|---------------------------------------|
| 0.01          | 0.635                                 |
| 0.05          | 3.79                                  |
| **0.1**       | **9.81**                              |
| 0.2           | 4.44                                  |
| 0.5           | 8.51                                  |

**Here is the FC agent I'll be keeping :**

In [58]:
agent = DQN_FC1(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
print('Training the mouse...')
train(agent,env,epochs_train,prefix='fc_train',display_loss=True)
print('...Done.')
print('Testing the mouse:')
test(agent,env,epochs_test,prefix='fc_test',display_score=True)
HTML(display_videos('fc_test20.mp4'))

Training the mouse...
Epoch 001/100 | Loss 1.6475 | Win/lose count 7.0/2.0 (5.0)
Epoch 002/100 | Loss 1.5679 | Win/lose count 4.5/0 (4.5)
Epoch 003/100 | Loss 1.6102 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/100 | Loss 1.5671 | Win/lose count 3.5/4.0 (-0.5)
Epoch 005/100 | Loss 1.6150 | Win/lose count 3.5/0 (3.5)
Epoch 006/100 | Loss 1.7689 | Win/lose count 3.5/1.0 (2.5)
Epoch 007/100 | Loss 1.7770 | Win/lose count 5.5/2.0 (3.5)
Epoch 008/100 | Loss 1.6862 | Win/lose count 8.5/5.0 (3.5)
Epoch 009/100 | Loss 1.7571 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/100 | Loss 1.7987 | Win/lose count 5.5/3.0 (2.5)
Epoch 011/100 | Loss 1.7834 | Win/lose count 4.0/1.0 (3.0)
Epoch 012/100 | Loss 1.7918 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/100 | Loss 1.6987 | Win/lose count 4.5/1.0 (3.5)
Epoch 014/100 | Loss 1.7292 | Win/lose count 4.5/2.0 (2.5)
Epoch 015/100 | Loss 1.6692 | Win/lose count 7.5/3.0 (4.5)
Epoch 016/100 | Loss 1.5857 | Win/lose count 6.5/2.0 (4.5)
Epoch 017/100 | Loss 1.6253 | Win/los

Epoch: 35, win/lose count 5.5/1.0. Average score (10.357142857142858)
Epoch: 36, win/lose count 10.0/0. Average score (10.347222222222221)
Epoch: 37, win/lose count 26.0/2.0. Average score (10.716216216216216)
Epoch: 38, win/lose count 14.0/2.0. Average score (10.75)
Epoch: 39, win/lose count 16.5/0. Average score (10.897435897435898)
Epoch: 40, win/lose count 12.0/2.0. Average score (10.875)
Epoch: 41, win/lose count 10.0/2.0. Average score (10.804878048780488)
Epoch: 42, win/lose count 6.0/4.0. Average score (10.595238095238095)
Epoch: 43, win/lose count 17.0/2.0. Average score (10.69767441860465)
Epoch: 44, win/lose count 16.5/1.0. Average score (10.806818181818182)
Epoch: 45, win/lose count 16.5/4.0. Average score (10.844444444444445)
Epoch: 46, win/lose count 10.0/2.0. Average score (10.782608695652174)
Epoch: 47, win/lose count 8.5/2.0. Average score (10.691489361702128)
Epoch: 48, win/lose count 9.5/0. Average score (10.666666666666666)
Epoch: 49, win/lose count 16.0/4.0. Averag

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [35]:
class DQN_CNN1(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN1, self).__init__(*args,**kwargs)
        
        # CNN model
        model = Sequential([Conv2D(30, kernel_size=(2,2), input_shape=(5, 5, self.n_state)),
            BatchNormalization(),
            Flatten(),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [44]:
class DQN_CNN2(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN2, self).__init__(*args,**kwargs)
        
        # CNN model
        model = Sequential([Conv2D(30, kernel_size=(2,2), input_shape=(5, 5, self.n_state)),
            BatchNormalization(),
            Conv2D(15, kernel_size=(2,2)),
            BatchNormalization(),
            Flatten(),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [13]:
class DQN_CNN3(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN3, self).__init__(*args,**kwargs)
        
        # CNN model
        model = Sequential([Conv2D(30, kernel_size=(2,2), input_shape=(5, 5, self.n_state)),
            BatchNormalization(),
            Conv2D(30, kernel_size=(2,2)),
            BatchNormalization(),
            Conv2D(15, kernel_size=(2,2)),
            BatchNormalization(),
            Flatten(),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [48]:
class DQN_CNN4(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN4, self).__init__(*args,**kwargs)
        
        # CNN model
        model = Sequential([Conv2D(30, kernel_size=(2,2), input_shape=(5, 5, self.n_state)),
            BatchNormalization(),
            Conv2D(60, kernel_size=(2,2)),
            BatchNormalization(),
            Conv2D(120, kernel_size=(2,2)),
            BatchNormalization(),
            Flatten(),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [50]:
class DQN_CNN5(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN5, self).__init__(*args,**kwargs)
        
        # CNN model
        model = Sequential([Conv2D(30, kernel_size=(2,2), input_shape=(5, 5, self.n_state)),
            BatchNormalization(),
            Conv2D(60, kernel_size=(2,2)),
            BatchNormalization(),
            Flatten(),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

Tests made with the following parameters **fixed** <br>
Learning Rate: 0.1, Memory: 2000, Batch Size: 32 

| Model used | Averaged Test Score (Over 100 epochs) |
|------------|---------------------------------------|
| DQN_CNN1   | -0.98                                 |
| DQN_CNN2   | -1.00                                 |
|**DQN_CNN3**| **9.6**                               |
| DQN_CNN4   | -0.90                                 |
| DQN_CNN5   | -0.81                                 |

**Trying to improve the performance by increasing batch size**

Tests made with the following parameters **fixed** <br>
Model: DQN_CNN3, Memory: 2000, Learning Rate: 0.1 

| Batch Size | Averaged Test Score (Over 100 epochs) |
|------------|---------------------------------------|
| **32**     | **9.6**                               |
| 64         | 7.31                                  |
| 256        | 5.39                                  |

**Trying different memory size**

Tests made with the following parameters **fixed** <br>
Model: DQN_CNN3, Learning Rate: 0.1, Batch Size: 32 

| Memory | Averaged Test Score (Over 100 epochs) |
|--------|---------------------------------------|
| 250    | 0.915                                 |
| 500    | 6.535                                 |
| 1000   | 5.53                                  |
| 2000   | 9.60                                  |
|**5000**| **11.83**                             |

**Trying different learning rates**

Tests made with the following parameters **fixed** <br>
Model: DQN_CNN3, Memory: 5000, Batch Size: 32 

| Learning Rate | Averaged Test Score (Over 100 epochs) |
|---------------|---------------------------------------|
| 0.01          | 10.69                                 |
| 0.05          | 7.80                                  |
| **0.1**       | **11.83**                             |
| 0.2           | -0.91                                 |
| 0.5           | -1.10                                 |

***

Here is the CNN agent I'll be keeping : 

In [88]:
agent = DQN_CNN3(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 32)
print('Training the mouse...')
train(agent,env,epochs_train,prefix='cnn_train',display_loss=True)
print('...Done.')
print('Testing the mouse:')
test(agent,env,epochs_test,prefix='cnn_test',display_score=True)
HTML(display_videos('cnn_test20.mp4'))

Training the mouse...
Epoch 001/100 | Loss 0.0421 | Win/lose count 6.5/10.0 (-3.5)
Epoch 002/100 | Loss 0.0753 | Win/lose count 1.5/4.0 (-2.5)
Epoch 003/100 | Loss 0.0485 | Win/lose count 2.0/5.0 (-3.0)
Epoch 004/100 | Loss 0.0323 | Win/lose count 2.0/5.0 (-3.0)
Epoch 005/100 | Loss 0.2320 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/100 | Loss 0.2091 | Win/lose count 4.0/4.0 (0.0)
Epoch 007/100 | Loss 0.4633 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/100 | Loss 0.9421 | Win/lose count 2.5/9.0 (-6.5)
Epoch 009/100 | Loss 2.3085 | Win/lose count 3.0/10.0 (-7.0)
Epoch 010/100 | Loss 2.2331 | Win/lose count 6.5/3.0 (3.5)
Epoch 011/100 | Loss 2.1865 | Win/lose count 5.5/2.0 (3.5)
Epoch 012/100 | Loss 2.1577 | Win/lose count 6.5/1.0 (5.5)
Epoch 013/100 | Loss 1.9091 | Win/lose count 10.5/1.0 (9.5)
Epoch 014/100 | Loss 1.9652 | Win/lose count 8.5/0 (8.5)
Epoch 015/100 | Loss 1.8736 | Win/lose count 8.0/0 (8.0)
Epoch 016/100 | Loss 1.9354 | Win/lose count 5.5/2.0 (3.5)
Epoch 017/100 | Loss 1.7461 |

Epoch: 36, win/lose count 9.0/1.0. Average score (9.277777777777779)
Epoch: 37, win/lose count 9.0/1.0. Average score (9.243243243243244)
Epoch: 38, win/lose count 4.5/2.0. Average score (9.06578947368421)
Epoch: 39, win/lose count 11.5/1.0. Average score (9.102564102564102)
Epoch: 40, win/lose count 8.5/6.0. Average score (8.9375)
Epoch: 41, win/lose count 5.0/0. Average score (8.841463414634147)
Epoch: 42, win/lose count 11.0/1.0. Average score (8.869047619047619)
Epoch: 43, win/lose count 13.0/4.0. Average score (8.872093023255815)
Epoch: 44, win/lose count 8.5/2.0. Average score (8.818181818181818)
Epoch: 45, win/lose count 3.0/2.0. Average score (8.644444444444444)
Epoch: 46, win/lose count 10.5/3.0. Average score (8.619565217391305)
Epoch: 47, win/lose count 15.5/4.0. Average score (8.680851063829786)
Epoch: 48, win/lose count 15.0/2.0. Average score (8.770833333333334)
Epoch: 49, win/lose count 13.5/2.0. Average score (8.826530612244898)
Epoch: 50, win/lose count 16.0/4.0. Avera

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [89]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN3(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC1(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test',display_score=True)
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test',display_score=True)

Test of the CNN
Epoch: 1, win/lose count 2.5/4.0. Average score (-1.5)
Epoch: 2, win/lose count 13.0/5.0. Average score (3.25)
Epoch: 3, win/lose count 20.5/4.0. Average score (7.666666666666667)
Epoch: 4, win/lose count 9.5/3.0. Average score (7.375)
Epoch: 5, win/lose count 7.5/3.0. Average score (6.8)
Epoch: 6, win/lose count 16.0/3.0. Average score (7.833333333333333)
Epoch: 7, win/lose count 17.0/1.0. Average score (9.0)
Epoch: 8, win/lose count 10.5/1.0. Average score (9.0625)
Epoch: 9, win/lose count 12.5/6.0. Average score (8.777777777777779)
Epoch: 10, win/lose count 8.0/1.0. Average score (8.6)
Epoch: 11, win/lose count 9.0/1.0. Average score (8.545454545454545)
Epoch: 12, win/lose count 13.5/2.0. Average score (8.791666666666666)
Epoch: 13, win/lose count 6.5/2.0. Average score (8.461538461538462)
Epoch: 14, win/lose count 12.5/3.0. Average score (8.535714285714286)
Epoch: 15, win/lose count 8.5/2.0. Average score (8.4)
Epoch: 16, win/lose count 6.0/0. Average score (8.25)
E

Epoch: 26, win/lose count 8.0/5.0. Average score (11.307692307692308)
Epoch: 27, win/lose count 17.5/1.0. Average score (11.5)
Epoch: 28, win/lose count 15.5/3.0. Average score (11.535714285714286)
Epoch: 29, win/lose count 11.0/1.0. Average score (11.482758620689655)
Epoch: 30, win/lose count 5.0/4.0. Average score (11.133333333333333)
Epoch: 31, win/lose count 2.0/0. Average score (10.838709677419354)
Epoch: 32, win/lose count 12.0/4.0. Average score (10.75)
Epoch: 33, win/lose count 12.0/5.0. Average score (10.636363636363637)
Epoch: 34, win/lose count 13.5/2.0. Average score (10.661764705882353)
Epoch: 35, win/lose count 15.5/3.0. Average score (10.714285714285714)
Epoch: 36, win/lose count 12.0/3.0. Average score (10.666666666666666)
Epoch: 37, win/lose count 6.0/1.0. Average score (10.513513513513514)
Epoch: 38, win/lose count 10.0/1.0. Average score (10.473684210526315)
Epoch: 39, win/lose count 19.5/0. Average score (10.705128205128204)
Epoch: 40, win/lose count 16.5/2.0. Avera

In [90]:
HTML(display_videos('cnn_test98.mp4'))

In [91]:
HTML(display_videos('fc_test99.mp4'))

**Changing the temperature**

In [92]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN3(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC1(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test',display_score=False)
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test',display_score=False)


HTML(display_videos('fc_test99.mp4'))

Test of the CNN
Final score: 2.83
Test of the FC
Final score: 3.53


In [34]:
#HTML(display_videos('fc_test99.mp4'))
HTML(display_videos('cnn_test98.mp4'))

In [93]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN3(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC1(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test',display_score=False)
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test',display_score=False)

Test of the CNN
Final score: 16.58
Test of the FC
Final score: 18.9


In [34]:
#HTML(display_videos('fc_test99.mp4'))
HTML(display_videos('cnn_test98.mp4'))

In [94]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN3(size, lr=.1, epsilon = 0.1, memory_size=5000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC1(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test',display_score=False)
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test',display_score=False)

Test of the CNN
Final score: 39.615
Test of the FC
Final score: 40.005


In [34]:
#HTML(display_videos('fc_test99.mp4'))
HTML(display_videos('cnn_test98.mp4'))

Both of the agent have the same behoavior regarding exploration : when their local environment becomes empty of bonuses, they oscillate and stop exploring. This is because their only mean of exploration is through a random taking a random action with probability <code>espilon</code> at each time step, which is almost always not enough to get out of the no-bonus zone. <br>
I am a bit surprised the CNN doesn't perform better than the one layer FC DQN. The CNN DQN always perform very slightly less good than the FC DQN. <br>
Finally, by changing the temperature, we see that, without surprise, both DQN get a higher average score if the temperature increase, because bonuses are abundant. When the temperature is low, bonuses and maluses are scarce and the agents, both the CNN one and the FC one, oscillate after a short amount of time in each epoch.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [16]:
def train_explore(agent,env,epochs,epsilon_decay,prefix='',display_loss=False):
    # Same function as train but we add the decaying of epsilon
    # Number of won games
    score = 0
    loss = 0
    
    # At the beginning of each epoch, we decay epsilon by the decay parameter
    # epsilon is multiplied by epsilon_decay
    for e in range(epochs):
        if e>0:
            agent.set_epsilon(epsilon_decay*agent.epsilon)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e+1))
            
        if display_loss:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                  .format(e+1, epochs, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        # This accounts for a visited state. We get a malus if the mouse goes there
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
    # We only need to rewrite 2 functions from Environment:
    # The act function and the reset function
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #reward = self.board[self.x, self.y] --> previously in Environment
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # Previously in Environment:
        #state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
        #                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # Previously in Environment:
        #state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        #self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        # 3 "feature" states instead of 2
        # You will have to change n_state to 3 because you will use one more layer!
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [27]:
# Training DQN
# memory_size=5000 crashed so I used 2000
epochs_train = 500
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN3(size, lr=.1, epsilon = 0.4, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent,env,epochs_train, epsilon_decay=0.95,prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

In [28]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore',display_score=True)

Epoch: 1, win/lose count 19.0/0. Average score (19.0)
Epoch: 2, win/lose count 4.0/0. Average score (11.5)
Epoch: 3, win/lose count 15.5/0. Average score (12.833333333333334)
Epoch: 4, win/lose count 10.5/0. Average score (12.25)
Epoch: 5, win/lose count 11.5/0. Average score (12.1)
Epoch: 6, win/lose count 14.5/0. Average score (12.5)
Epoch: 7, win/lose count 5.5/0. Average score (11.5)
Epoch: 8, win/lose count 18.0/0. Average score (12.3125)
Epoch: 9, win/lose count 13.5/0. Average score (12.444444444444445)
Epoch: 10, win/lose count 12.0/0. Average score (12.4)
Epoch: 11, win/lose count 8.5/0. Average score (12.045454545454545)
Epoch: 12, win/lose count 6.0/0. Average score (11.541666666666666)
Epoch: 13, win/lose count 9.0/0. Average score (11.346153846153847)
Epoch: 14, win/lose count 8.0/0. Average score (11.107142857142858)
Epoch: 15, win/lose count 17.5/0. Average score (11.533333333333333)
Epoch: 16, win/lose count 10.5/0. Average score (11.46875)
Epoch: 17, win/lose count 9.0

In [29]:
HTML(display_videos('cnn_test_explore3.mp4'))

In [30]:
# Training FC
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC1(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent,env,epochs_train, epsilon_decay=0.9,prefix='fc_train_explore')
HTML(display_videos('fc_train_explore10.mp4'))

In [31]:
# Evaluation
test(agent,env,epochs_test,prefix='fc_test_explore',display_score=True)

Epoch: 1, win/lose count 5.5/0. Average score (5.5)
Epoch: 2, win/lose count 13.0/0. Average score (9.25)
Epoch: 3, win/lose count 17.0/0. Average score (11.833333333333334)
Epoch: 4, win/lose count 13.5/0. Average score (12.25)
Epoch: 5, win/lose count 11.5/0. Average score (12.1)
Epoch: 6, win/lose count 16.5/0. Average score (12.833333333333334)
Epoch: 7, win/lose count 19.0/0. Average score (13.714285714285714)
Epoch: 8, win/lose count 19.5/0. Average score (14.4375)
Epoch: 9, win/lose count 15.0/0. Average score (14.5)
Epoch: 10, win/lose count 12.5/0. Average score (14.3)
Epoch: 11, win/lose count 9.0/0. Average score (13.818181818181818)
Epoch: 12, win/lose count 7.0/0. Average score (13.25)
Epoch: 13, win/lose count 11.5/0. Average score (13.115384615384615)
Epoch: 14, win/lose count 5.0/0. Average score (12.535714285714286)
Epoch: 15, win/lose count 3.5/0. Average score (11.933333333333334)
Epoch: 16, win/lose count 14.0/0. Average score (12.0625)
Epoch: 17, win/lose count 4.5

In [32]:
HTML(display_videos('fc_test_explore10.mp4'))

I tried different parameters for the exploration task. I went from <code>epochs_train=100</code>, <code>epsilon=0.5</code>, <code>epsilon_decay=0.9</code> to <code>epochs_train=100</code>, <code>epsilon=0.1</code>, <code>epsilon_decay=0.99</code> which took a lot of time to try to have a good exploration. I settled on increasing the training time to <code>epochs_train=500</code>, starting at a medium-low <code>epsilon=0.2</code> and a slow decay <code>epsilon_decay=0.95</code>. <br>
The exploration seems to make the test score a bit better. I tried both the FC and CNN DQN and found by looking through the videos that the exploration doesn't seem to add a lot to the FC, but it seems that with the CNN, the agent does try to go further and not where it has already been, in order to collect more bonuses.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***